In [1]:
import tensorflow as tf
import numpy as np
from spiking_models import DenseRNN, SpikingReLU, SpikingSigmoid, SpikingTanh, Accumulate
import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Input, Model

In [2]:
def convert(model, weights, x_test, y_test):
    print("Converted model:\n" + "-"*32)
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.InputLayer):
            print("Input Layer")
            inputs = tf.keras.Input(shape=(1, model.layers[0].input_shape[0][1]), batch_size=y_test.shape[0])
            x = inputs
        elif isinstance(layer, tf.keras.layers.Dense):
            x = tf.keras.layers.Dense(layer.output_shape[1])(x)
            # x = tf.keras.layers.RNN(DenseRNN(layer.output_shape[1]), return_sequences=True, return_state=False, stateful=True)(x)
            if layer.activation.__name__ == 'linear':
                print("Dense Layer w/o activation")
                pass
            elif layer.activation.__name__ == 'relu':
                print("Dense Layer with SpikingReLU")
                x = tf.keras.layers.RNN(SpikingReLU(layer.output_shape[1]), return_sequences=True, return_state=False, stateful=True)(x)
            elif layer.activation.__name__ == 'sigmoid':
                print("Dense Layer with SpikingSigmoid")
                x = tf.keras.layers.RNN(SpikingSigmoid(layer.output_shape[1]), return_sequences=True, return_state=False, stateful=True)(x)
            elif layer.activation.__name__ == 'tanh':
                print("Dense Layer with SpikingTanh")
                x = tf.keras.layers.RNN(SpikingTanh(layer.output_shape[1]), return_sequences=True, return_state=False, stateful=True)(x)
            else:
                print('[Info] Activation type', layer.activation.__name__, 'not implemented')
        elif isinstance(layer, tf.keras.layers.ReLU):
            print("SpikingReLU Layer")
            x = tf.keras.layers.RNN(SpikingReLU(layer.output_shape[1]), return_sequences=True, return_state=False, stateful=True)(x)
        elif isinstance(layer, tf.keras.layers.Softmax):
            print("Accumulate + Softmax Layer")
            print(layer.output_shape[1])
            x = tf.keras.layers.RNN(Accumulate(layer.output_shape[1]), return_sequences=True, return_state=False, stateful=True)(x)
            x = tf.keras.layers.Softmax()(x)
        else:
            print("[Info] Layer type ", layer, "not implemented")
    spiking = tf.keras.models.Model(inputs=inputs, outputs=x)
    print("-"*32 + "\n")

    spiking.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer="adam",
        metrics=["sparse_categorical_accuracy"],)

    spiking.set_weights(weights)
    return spiking

In [3]:
def get_normalized_weights(model, x_test, percentile=100):
    max_activation = 0
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.ReLU):
            activation = tf.keras.Model(inputs=model.inputs, outputs=layer.output)(x_test).numpy()
            if np.percentile(activation, percentile) > max_activation:
                max_activation = np.percentile(activation, percentile)
        elif isinstance(layer, tf.keras.layers.Dense):
            if layer.activation.__name__ == 'relu':
                activation = tf.keras.Model(inputs=model.inputs, outputs=layer.output)(x_test).numpy()
                if np.percentile(activation, percentile) > max_activation:
                    max_activation = np.percentile(activation, percentile)

    weights = model.get_weights()
    if max_activation == 0:
        print("\n" + "-"*32 + "\nNo normalization\n" + "-"*32)
    else:
        print("\n" + "-"*32 + "\nNormalizing by", max_activation, "\n" + "-"*32)
        for i in range(len(weights)):
            weights[i] /= (max_activation)
    return weights


def evaluate_conversion(converted_model, original_model, x_test, y_test, testacc, timesteps=50):
    for i in range(1, timesteps+1):
        _, acc = converted_model.evaluate(x_test, y_test, batch_size=y_test.shape[0], verbose=0)
        print(
            "Timesteps", str(i) + "/" + str(timesteps) + " -",
            "acc spiking (orig): %.2f%% (%.2f%%)" % (acc*100, testacc*100),
            "- conv loss: %+.2f%%" % ((-(1 - acc/testacc)*100)))

In [4]:
def weight_conversion_model(weights, bias):
    """
    Simple model-based conversion model proposed by Diehl et al.
    :param weights: weights of the network.
    :param bias: bias of the network.
    :return: rescaled weights.
    """
    # Get weights from trained network
    converted_weights = weights
    converted_bias = bias

    # model based normalization
    previous_factor = 1
    for l in range(len(converted_weights)):
        max_pos_input = 0
        # Find maximum input for this layer
        for o in range(converted_weights[l].shape[0]):
            input_sum = 0
            for i in range(converted_weights[l].shape[1]):
                input_sum += tf.math.maximum(0, converted_weights[l][o, i])
            if converted_bias is not None and converted_bias[l] is not None:
                input_sum += tf.math.maximum(0, converted_bias[l][o])
            max_pos_input = tf.math.maximum(max_pos_input, input_sum)

        # get the maximum weight in the layer, in case all weights are negative, max_pos_input would be zero, so we
        # use the max weight to rescale instead
        max_wt = tf.math.reduce_max(converted_weights[l])
        if converted_bias is not None and converted_bias[l] is not None:
            max_bias = tf.math.reduce_max(converted_bias[l])
            max_wt = tf.math.maximum(max_wt, max_bias)
        scale_factor = tf.math.maximum(max_wt, max_pos_input)
        # Rescale all weights
        applied_factor = scale_factor / previous_factor
        converted_weights[l] = converted_weights[l] / applied_factor
        if converted_bias is not None and converted_bias[l] is not None:
            converted_bias[l] = converted_bias[l] / scale_factor
        previous_factor = scale_factor
        print(f"Scale factor for this layer is {previous_factor}")

    return converted_weights, converted_bias


def weight_conversion_robust_and_data_based(weights, bias, model, data, normalization_method='robust',
                                            ppercentile=0.99):

    """
    Two methods proposed by Diehl et al and Rueckauer et al. Both methods are data-based, so they use weights and
    activations to find the best scaling factor.
    :param weights: weights of the network.
    :param bias: bias of the network.
    :param model: ann model.
    :param data: dataset to determine activations.
    :param normalization_method: type of normalization - robust (Rueckauer) or data (Diehl).
    :param ppercentile: percentile of the activation, which is taken from maximal activation.
    :return: rescaled weights.
    """
    if normalization_method == 'data':
        ppercentile = 1.0

    # Get weights from trained network
    converted_weights = weights
    converted_bias = bias

    # use training set to find max_act for each neuron

    activations = []
    for l in range(0, len(converted_weights)):
        activation = get_activations_layer(model.input, model.layers[l].output, data)
        activation_per_neuron = [np.max(activation[:, i]) for i in range(activation.shape[1])]
        activations.append(activation_per_neuron)

    previous_factor = 1
    for l in range(len(converted_weights)):
        # get the p-percentile of the activation
        pos_inputs = activations[l]
        pos_inputs.sort()
        max_act = pos_inputs[int(ppercentile * (len(pos_inputs) - 1))]
        # get the maximum weight in the layer
        max_wt = tf.math.reduce_max(converted_weights[l])
        if converted_bias is not None and converted_bias[l] is not None:
            max_bias = tf.math.reduce_max(converted_bias[l])
            max_wt = tf.math.maximum(max_wt, max_bias)
        scale_factor = tf.math.maximum(max_wt, max_act)

        applied_factor = scale_factor / previous_factor
        # rescale weights
        converted_weights[l] = converted_weights[l] / applied_factor

        # rescale bias
        if converted_bias is not None and converted_bias[l] is not None:
            converted_bias[l] = converted_bias[l] / scale_factor
        previous_factor = scale_factor
        print(f"Scale factor for this layer is {previous_factor}")

    return converted_weights, converted_bias


def get_activations_layer(layer_in, layer_out, data, batch_size=32):

    """
    Getting activation for specific layer of neural network.
    :param layer_in: input layer of a model. For sequential models first layer, for functional model.layers[0].input can
    be used.
    :param layer_out: layer for which activations should be computed. For functional model.layers[i].output can be used.
    :param data: dataset.
    :param batch_size: batch_size of batches in which dataset should be divided.
    :return: activations for a specific layer for all
    """

    if len(data) % batch_size != 0:
        data = data[: -(len(data) % batch_size)]

    return Model(layer_in, layer_out).predict(data, batch_size)

In [5]:
tf.random.set_seed(1234)
batch_size=512
epochs = 5
act='relu'


def create_ann():
    inputs = tf.keras.Input(shape=(784,))
    x = tf.keras.layers.Dense(500, activation=act)(inputs)
    #x = tf.keras.layers.ReLU()(x)  # max_value=1
    x = tf.keras.layers.Dense(100, activation=act)(x)
    #x = tf.keras.layers.Activation(tf.nn.relu)(x)  # not implemented yet
    x = tf.keras.layers.Dense(10, activation=act)(x)
    x = tf.keras.layers.Softmax()(x)
    ann = tf.keras.Model(inputs=inputs, outputs=x)

    ann.compile(
        optimizer=tf.keras.optimizers.RMSprop(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

    ann.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs)
    return ann

In [6]:
##################################################
# Import Data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255., x_test / 255.
x_train = x_train.reshape((60000, 784))
x_test = x_test.reshape((10000, 784))

# Analog model
ann = create_ann()

# _, testacc = ann.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)
# #weights = ann.get_weights()
# weights = get_normalized_weights(ann, x_train, percentile=85)

# ##################################################
# # Preprocessing for RNN
# x_train = np.expand_dims(x_train, axis=1)  # (60000, 784) -> (60000, 1, 784)
# x_test = np.expand_dims(x_test, axis=1)
# #x_rnn = np.tile(x_train, (1, 1, 1))
# #y_rnn = y_train  # np.tile(x_test, (1, timesteps, 1))

# ##################################################
# # Conversion to spiking model
# snn = convert(ann, weights, x_test, y_test)
# evaluate_conversion(snn, ann, x_test, y_test, testacc, timesteps=50)

Epoch 1/5
118/118 [==============================] - 1s 6ms/step - loss: 0.7013 - sparse_categorical_accuracy: 0.7837
Epoch 2/5
118/118 [==============================] - 1s 6ms/step - loss: 0.1656 - sparse_categorical_accuracy: 0.9499
Epoch 3/5
118/118 [==============================] - 1s 6ms/step - loss: 0.1091 - sparse_categorical_accuracy: 0.9670
Epoch 4/5
118/118 [==============================] - 1s 6ms/step - loss: 0.0720 - sparse_categorical_accuracy: 0.9773
Epoch 5/5
118/118 [==============================] - 1s 5ms/step - loss: 0.0562 - sparse_categorical_accuracy: 0.9831


In [7]:
weights_original = ann.get_weights()

In [9]:
ann.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 500)               392500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               50100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
_________________________________________________________________
softmax (Softmax)            (None, 10)                0         
Total params: 443,610
Trainable params: 443,610
Non-trainable params: 0
_________________________________________________________________


In [41]:
weights = np.array([weights_original[0], weights_original[2], weights_original[4]])
bias = np.array([weights_original[1], weights_original[3], weights_original[5]])


from tensorflow.keras.models import Model
import tensorflow_datasets as tfds

def flatten(image, label):
    '''Transform image to the flattened version of itself'''
    flattened = tf.reshape(image, [28*28])
    flattened = tf.expand_dims(flattened, 0)
    return tf.cast(flattened, tf.float32), label

model = ann

(ds_train, ds_test) = tfds.load('mnist', split=['train', 'test'], shuffle_files=True, as_supervised=True)
ds_train = ds_train.map(flatten, num_parallel_calls=tf.data.experimental.AUTOTUNE)

result = get_activations_layer(model.input, model.layers[1].output, ds_train)


# converted_weights_data_or_robust = weight_conversion_robust_and_data_based(weights, bias, model, ds_train)

converted_weights_model = weight_conversion_model(weights, bias)

### Applying weights to the architecture

# converted_weights = weight_conversion(weights)

# expanded_converted_wt_0 = tf.expand_dims(converted_weights[0], 0)
# expanded_converted_wt_1 = tf.expand_dims(converted_weights[1], 0)

# model.layers[1].set_weights(expanded_converted_wt_0)
# model.layers[2].set_weights(expanded_converted_wt_1)


# model.evaluate(ds_test)

# expanded_wt_0 = tf.expand_dims(weights[0], 0)
# expanded_wt_1 = tf.expand_dims(weights[2], 0)

# model.layers[1].set_weights(expanded_wt_0)
# model.layers[2].set_weights(expanded_wt_1)

# model.layers[1].get_weights()[1]

# model.evaluate(ds_test)

# random_wt_0 = tf.random.normal(weights[0].shape)
# random_wt_0 = tf.expand_dims(random_wt_0, 0)

# random_wt_1 = tf.random.normal(weights[2].shape)
# random_wt_1 = tf.expand_dims(random_wt_1, 0)

# model.layers[1].set_weights(random_wt_0)
# model.layers[2].set_weights(random_wt_1)

# model.evaluate(ds_test)

<ipython-input-41-d7061c597c31>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  weights = np.array([weights_original[0], weights_original[2], weights_original[4]])
<ipython-input-41-d7061c597c31>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  bias = np.array([weights_original[1], weights_original[3], weights_original[5]])


IndexError: index 500 is out of bounds for axis 0 with size 500

In [39]:
converted_weights_data_or_robust[1][1]

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([ 1.2715519e-05, -1.0592952e-05,  2.0176419e-06,  3.0930821e-06,
       -5.3669709e-07,  2.4352505e-05, -2.1009200e-05,  1.2117722e-05,
       -3.1248940e-06,  7.6323986e-06,  7.8802432e-06, -4.1792882e-06,
        8.4965059e-06, -4.2733741e-06,  5.0357944e-06, -1.2094863e-05,
        2.3230412e-05,  2.7503418e-06,  7.2594089e-06,  2.6647469e-05,
        2.2722406e-05,  6.2220679e-06,  2.4737466e-05,  1.2531269e-05,
       -1.3302352e-05,  1.7394526e-05,  1.2308599e-05,  1.3999711e-05,
        3.6008103e-06,  2.2414417e-05,  8.9935929e-06,  4.0104916e-07,
        2.7079677e-05,  8.0010077e-06,  7.9747588e-06,  9.2475702e-06,
        1.6139924e-05,  1.0039097e-05,  8.7524704e-06,  1.3726043e-05,
        8.7641383e-06, -6.0516055e-07,  2.0943609e-05, -7.2720131e-06,
        6.4489186e-06,  8.1431508e-06,  2.1846565e-05,  2.9648354e-05,
        1.5644033e-05,  2.7622764e-05,  1.3171528e-05,  3.7484332e-07,
        6.3031948e-06, -8.272

In [40]:
bias[1]

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([ 1.2715519e-05, -1.0592952e-05,  2.0176419e-06,  3.0930821e-06,
       -5.3669709e-07,  2.4352505e-05, -2.1009200e-05,  1.2117722e-05,
       -3.1248940e-06,  7.6323986e-06,  7.8802432e-06, -4.1792882e-06,
        8.4965059e-06, -4.2733741e-06,  5.0357944e-06, -1.2094863e-05,
        2.3230412e-05,  2.7503418e-06,  7.2594089e-06,  2.6647469e-05,
        2.2722406e-05,  6.2220679e-06,  2.4737466e-05,  1.2531269e-05,
       -1.3302352e-05,  1.7394526e-05,  1.2308599e-05,  1.3999711e-05,
        3.6008103e-06,  2.2414417e-05,  8.9935929e-06,  4.0104916e-07,
        2.7079677e-05,  8.0010077e-06,  7.9747588e-06,  9.2475702e-06,
        1.6139924e-05,  1.0039097e-05,  8.7524704e-06,  1.3726043e-05,
        8.7641383e-06, -6.0516055e-07,  2.0943609e-05, -7.2720131e-06,
        6.4489186e-06,  8.1431508e-06,  2.1846565e-05,  2.9648354e-05,
        1.5644033e-05,  2.7622764e-05,  1.3171528e-05,  3.7484332e-07,
        6.3031948e-06, -8.272